<a href="https://colab.research.google.com/github/Rogerio-mack/IA_2024S2/blob/main/Mack_Lab_regressoes_solucao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="Universidade Presbiteriana Mackenzie">
</head>

<img src="http://meusite.mackenzie.br/rogerio/mackenzie_logo/UPM.2_horizontal_vermelho.jpg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->


# **Lab: Regressão Linear e Logística**

## imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.linear_model import LogisticRegression
import statsmodels.formula.api as sm


In [ ]:
#@markdown Data Preparation - **Just Run This Code**
df = sns.load_dataset('penguins')
df = df.dropna()
df.to_csv('penguins.csv', index=False)

df_case = pd.DataFrame(df[ (df.species == 'Adelie') & (df.sex == 'Female') ].select_dtypes(include='number').drop(columns='body_mass_g').mean()).T
df_case['species'] = 'Adelie'
df_case['island'] = 'Torgersen'
df_case['sex'] = 'Female'
df_case = df_case[['species', 'island', 'bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'sex']]
df_case.to_csv('penguins_case.csv', index=False)

In [ ]:
df = pd.read_csv('penguins.csv')
df_case = pd.read_csv('penguins_case.csv')

display(df.head())
display(df_case.head())

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
3,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female
4,Adelie,Torgersen,39.3,20.6,190.0,3650.0,Male


,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,sex
0,Adelie,Torgersen,37.257534,17.621918,187.794521,Female


# Ex1. `statsmodels`

Faça um modelo de regressão linear para estimar os valores de `body_mass_g` com base em **todos os atributos preditores SIGNIFICATIVOS**. Com o modelo  obtido faça a predição do peso (`body_mass_g`) do pinguim de `df_case`.

<br>

**Dica**

1. Faça o modelo com todos os atributos preditores e depois elimine os não significativos.


In [ ]:
lm = sm.ols('body_mass_g ~ species + island + bill_length_mm + bill_depth_mm + flipper_length_mm + sex', data=df).fit()
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:            body_mass_g   R-squared:                       0.875
Model:                            OLS   Adj. R-squared:                  0.872
Method:                 Least Squares   F-statistic:                     284.1
Date:                Tue, 10 Sep 2024   Prob (F-statistic):          1.85e-141
Time:                        13:17:14   Log-Likelihood:                -2353.6
No. Observations:                 333   AIC:                             4725.
Df Residuals:                     324   BIC:                             4760.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept            -1500.0291 

In [ ]:
lm = sm.ols('body_mass_g ~ species + bill_length_mm + bill_depth_mm + flipper_length_mm + sex', data=df).fit()
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:            body_mass_g   R-squared:                       0.875
Model:                            OLS   Adj. R-squared:                  0.873
Method:                 Least Squares   F-statistic:                     380.2
Date:                Tue, 10 Sep 2024   Prob (F-statistic):          6.82e-144
Time:                        13:17:14   Log-Likelihood:                -2354.0
No. Observations:                 333   AIC:                             4722.
Df Residuals:                     326   BIC:                             4749.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept            -1460.9946 

In [ ]:
lm.predict(df_case).values


array([3397.13003713])

# Ex2. `scikit-learn`  

Faça um modelo de **regressão logística** para estimar a espécie do pinguim a partir unicamente dos preditores numéricos da base de dados. Empregue todos os dados para treinamento do modelo e verifique a acuracidade do modelo (também sobre todo os dados) e a predição para da espécie para o pinguim de `df_case` adicionando agora a informação de peso do pinguim, de 3400.


<br>

**Nota**

1. Empregue `max_iter=1000` na regressão logística.
2. Não separe conjuntos de treinamento e teste. Faremos isso somente mais adiante quando você entender por que deve fazer isso.
3. Para a predição não esqueça de adicionar o peso, `body_mass_g`, com o valor de 3400 ao `df_case`


In [ ]:
X = df.drop(['species', 'island', 'sex'], axis=1)
y = df['species']

# Criar o modelo de regressão logística
model = LogisticRegression(max_iter=1000)

# Treinar o modelo
model.fit(X,y)

# Fazer previsões no conjunto de teste
y_pred = model.predict(X)

# Calcular a acurácia do modelo
accuracy = model.score(X, y)
print("Acurácia do modelo:", accuracy)

Acurácia do modelo: 0.993993993993994


In [ ]:
df_case['body_mass_g'] = 3400
df_case.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,sex,body_mass_g
0,Adelie,Torgersen,37.257534,17.621918,187.794521,Female,3400


In [ ]:
model.predict(df_case.drop(['species', 'island', 'sex'], axis=1))

array(['Adelie'], dtype=object)